# Real Quantum Amplitude Estimation (RQAE) module

Present notebook reviews the **Real Quantum Amplitude Estimation** (RQAE) algorithms which were implemented into the module *real_quantum_ae* within the package *AE* of library *QQuantLib* (**QQuantLib/AE/real_quantum_ae.py**).

Present notebook and module are based on the following references:


* *Manzano, A., Musso, D., Leitao, A., Gómez, A., Vázquez, C., Nogueiras, M. & Ordóñez, G.*. Real Quantum Amplitude Estimation. Preprint. https://arxiv.org/abs/2204.13641

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
%matplotlib inline

In [ ]:
#This cell loads the QLM solver.
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg (for using QPU as CESGA QLM one)
from QQuantLib.utils.qlm_solver import get_qpu
QLMaaS = False
linalg_qpu = get_qpu(QLMaaS)

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results
from QQuantLib.utils.utils import bitfield_to_int

## 1. Oracle generation

Before doing any amplitude estimation we want to load some data into the quantum circuit, as this step is only auxiliary to see how the algorithm works, we are just going to load a discrete probability distribution. In this case we will have a circuit with $n=3$ qubits which makes a total of $N = 2^n = 8$ states. The discrete probability distribution that we are going to load is:
$$p_d = \dfrac{(0,1,2,3,4,5,6,7)}{0+1+2+3+4+5+6+7+8}.$$


In [ ]:
n = 3
N = 2**n
x = np.arange(N)
probability = x/np.sum(x)

Note that this probability distribution is properly normalised. For loading this probability into the quantum circuit we will use the function *load_probability* from **QQuantLib/DL/data_loading** module. The state that we are going to get is:
    $$|\Psi\rangle = \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].$$

In [ ]:
from QQuantLib.DL.data_loading import load_probability

In [ ]:
oracle = load_probability(probability)

For more information about loading data into the quantum circuit see the notebook *01_DataLoading_Module_Use*.

## 2. RQAE class

### 2.1 The Amplitude Estimation Problem

The **RQAE** algorithm solves the **amplitude estimation** problem when a little variation is added.  In this case given an oracle:

$$\mathcal{0}|0\rangle = |\Psi\rangle = a|\Psi_0\rangle +\sqrt{1-a^2}|\Psi_1\rangle,$$

where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states, *we want to estimate the real parameter $a$ (so $a$ can take values in the domain $[-1,1]$)*

**BE AWARE** 

In Notebooks: *03_Maximum_Likelihood_Amplitude_Estimation_Class.ipynb*, *04_Classical_Phase_Estimation_Class.ipynb*, *05_Iterative_Quantum_Phase_Estimation_Class.ipynb*, *06_Iterative_Quantum_Amplitude_Estimation_class.ipynb* we want to estimate $\sqrt{a}$ meanwhile in this new problem we want to estimate $a$


### 2.2 RQAE algorithm output

Given an error $\epsilon$ and a confident interval $\gamma$, the **RQAE** algorithm allows to estimate the $a$, from the amplitude estimation problem presented in *Section 2.1*, satisfying:

$$P\big[a \in [a_l, a_u]\big] \gt 1-\gamma$$

and

$$\frac{a_u-a_l}{2} \leq \epsilon$$


### 2.3 Creating object from RQAE class

We have implemented and python class called **RQAE** into the **QQuantLib/AE/real_quantum_ae** module that allows us to use the **RQAE** algorithm.

For creating the **RQAE** class the conventions used in **MLAE class** from **QQuantLib/AE/maximum_likelihood_ae.py** module should be followed: 

We have some mandatory inputs:

1. Oracle: QLM AbstractGate or QRoutine with the implementation of the Oracle for creating the Grover operator.
2. target: this is the marked state in binnary representation as a python list. 
3. index: list of the qbits affected by the Grover operator. 

And some optional inputs, used for algorithm configuration, that can be given as a python dictionary:
* qpu: QLM solver tha will be used
* epsilon ($\epsilon$): the precision. Ensures that the width of the interval is (see Section 2.2), at most, $2\epsilon$ (default: 0.01).
* gamma ($\gamma$): the accuracy. Ensures that the probability of $a$ not laying within the given interval (see Section 2.2) is, at most, $\gamma$ (default: 0.05).
* ratio: the amplification ratio (default: 2).

In [ ]:
#import the class
from QQuantLib.AE.real_quantum_ae import RQAE

For showing how our **RQAE** class works, we will define the following amplitude estimation problem:

$$
    \begin{array}{l}
    &\mathcal{O}\longrightarrow \mathcal{P}.\\
    & |\Psi\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    & a|\Psi_0\rangle \longrightarrow \dfrac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle.\\
    & \sqrt{1-a^2}|\Psi_1\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    \end{array}
$$

The target state, in this case is $|1\rangle$. It's binary representation is $001$. This has to be passed to the target variable as a list. Moreover we have to provide the list of qubits where we are acting, in this case is just $[0,1,2]$, the whole register.

In [ ]:
target = [0,0,1]
index = [0,1,2]

#We want to estimate the probability of target.
#In the RQAE solutionc the a is codified as an amplitude not as a probability
a_real = np.sqrt(probability[bitfield_to_int(target)])

print('We want to estimate: ', a_real)

rqae_dict = {
    'qpu': linalg_qpu,
}

rqae = RQAE(oracle,target = [0,0,1],index = [0,1,2], **rqae_dict)

### 2.4 The *rqae* method

In order to execute the complete algorithm using the **RQAE** class the *rqae* method can be used. 

 This method has the following inputs:
* ratio: the amplification ratio
* epsilon ($\epsilon$): error in the estimation of $a$ (default: 0.01).
* gamma ($\gamma$): confidence interval for the $a$ estimation (default: 0.05).

This method returns the limits for the $a$ estimation: $(a_{\min},a_{\max})$

In [ ]:
epsilon = 0.01
gamma = 0.05
q = 2.0
bounds = rqae.rqae(ratio=q, epsilon=epsilon, gamma=gamma)

In [ ]:
print('Bounds for a: [a_l, a_u] = [{}, {}]'.format(bounds[0], bounds[1]))
a_estimated = (bounds[0]+bounds[1])/2.0
print('a_estimated: ', a_estimated)
print('a real: ', a_real)
print('|a_l-a_estimated| = ', np.abs(a_estimated-a_real))
print('Error estimation wanted: ', 0.01)

In [ ]:
if (a_real>=bounds[0])&(a_real<=bounds[1]):
    print("Correct")
else:
    print("Incorrect")

### 2.5 The *display_information* method

The display method gives some information of the inner working of the **RQAE** algorithm. The inputs are the same that for the *rqae* methdo.

In [ ]:
rqae.display_information(ratio = q,epsilon = epsilon, gamma = gamma)

### 2.6 The *run* method

Finally a *run* method for a direct implementation of the **RQAE** algorithm was implemented. In this case the user can configure all the properties of the **RQAR** class and the *run* method will execute the method using the fixed attributes of the class. Finally the method returns the estimation of $a=\frac{a_u+a_l}{2}$. Additionally the *run* method populates following class attributes:

* ae_l: the lower limit for a $a_l$.
* ae_l: the upper limit for a $a_u$.
* ae: the amplitude estimation parameter calculated as: $a=\frac{a_u+a_l}{2}$


In [ ]:
#First we create the class
target = [0,0,1]
index = [0,1,2]
a = np.sqrt(probability[bitfield_to_int(target)])

print('We want to estimate: ', a)

epsilon = 0.001
q = 2
gamma = 0.01

rqae_dict = {
    'qpu': linalg_qpu,    
    'epsilon': epsilon,
    'ratio': q,
    'gamma': gamma
}

rqae_ = RQAE(oracle, target = target, index = [0,1,2], **rqae_dict)

In [ ]:
a_estimated = rqae_.run()

In [ ]:
print('a_estimated: ', a_estimated)
print('Real Value of a: ', a)
print('Bounds for a: [rqae.a_l, rqae.a_u] = [{}, {}]'.format(rqae_.ae_l, rqae_.ae_u))
print('Estimated a: rqae.a= ', rqae_.ae)
print('|a_l-a_estimated| = ', np.abs(a_real-rqae_.ae))
print('Error estimation wanted: ', rqae_.epsilon)